In [38]:
# top-down code to plan observing schedule for a LIGO event

import numpy as np
import pandas as pd

from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u

target_filename = "test.csv"
starting_time_jd = 2460068.33333
ending_time_jd = 2460068.66666

utc_offset = +1*u.hour

In [35]:
def conv_to_deg(deg, arcmin, arcsec):
    return (deg+ arcmin/60+ arcsec/3600)*u.deg

In [37]:
Wendelstein_loc = EarthLocation(lat = conv_to_deg(47, 42, 13.1), lon = conv_to_deg(12, 0, 43.4), height = 1838*u.m)

<EarthLocation (4207201.56230762, 895193.56849941, 4696123.67205851) m>

In [24]:

def read_target_list(filename):
    # read from a file
    file = pd.read_csv(filename)
    return file

target_list = read_target_list(target_filename)
# input from Julius
# has to contain: RA, dec, sersic fit to host galaxy, redshift, (relative) probability of being the host

In [25]:
target_list[:5]

,RA,dec,sersic,redshfit,probability
0,15,15,4,0.2,0.08


In [29]:
def initialize_observing_plan(target_list, starting_time_jd, ending_time_jd):
        # determine which targets are in any way observable that night, remove rest
        # order observable targets by their RA
        # return that list

        # delete all entries with dec lower than -20°, since Wendelstein can't observe that
        target_list = target_list[target_list["dec"]> -20]

        alt = []
        az = []

        target_list["Alt"] = alt
        target_list["Az"] = az

        print(target_list)

        return target_list

initial_observing_plan = initialize_observing_plan(target_list, starting_time_jd, ending_time_jd)
# re-ordered, sub-selected version of target list


   RA  dec  sersic  redshfit  probability
0  15   15       4       0.2         0.08
   alt
0    2
   az
0   1


In [17]:
def evaluate_plan(plan, starting_time_jd, ending_time_jd, alpha=0):
    # energy = sum_i p_i ((t_end - tobs_i)/(t_end - t_begin))^alpha
    # alpha = 0 is just maximizing total probability
    # alpha > 0 prioritizes getting things done quickly
    
    current_time = starting_time_jd
    current_RA = plan[0]['RA']
    current_dec = plan[0]['dec']
    
    schedule = []
    
    for target in plan:
        slewing_time  = slewing_time(current_RA, current_dec, target['RA'],target['dec'])
        exposure_time = determine_required_exposure_time(target,current_time)
        
        if(current_time + slewing_time + exposure_time < ending_time_jd): # we can observe this target
            schedule_entry = {}
            schedule_entry['observed'] = True
            schedule_entry['starting_time'] = current_time + slewing_time
            schedule_entry['exposure_time'] = exposure_time
            schedule.append(schedule_entry)
            current_time += slewing_time + exposure_time
            energy += target['probability']*((ending_time_jd - current_time)/(ending_time_jd - starting_time_jd))**alpha
            current_RA = target['RA']
            current_dec = target['dec']
        else:
            schedule_entry = {}
            schedule['observed'] = False
            schedule.append(schedule_entry)
            
    return energy,schedule

def change_plan(plan):
    
    # just implement a single atomic change: pick a target and move it to a different position in the plan
    
    return plan

def print_observing_plan(plan, schedule):
    print(plan, schedule)
    

def slewing_time(ra1, dec1, ra2, dec2):
    return 0 # actually determine angular separation ...

def determine_required_exposure_time(target, starting_time):
    return 1200/number_of_seconds_in_a_day # + readout time # Ananya's

In [20]:
temperature  = 100
cooling_rate = 0.99
iterations   = 100

current_observing_plan = np.copy(initial_observing_plan)
current_energy,current_schedule = evaluate_plan(current_observing_plan, starting_time_jd, ending_time_jd)

for i in range(iterations):
    print("iteration ",i)
    
    proposed_observing_plan = change_plan(current_observing_plan) # makes a random change
    proposed_energy,proposed_schedule = evaluate_plan(proposed_observing_plan, starting_time_jd, ending_time_jd)
    
    # should we accept the new plan?

    acceptance_probability = np.exp(-(current_energy-proposed_energy)/temperature) # >1 if proposed_energy>current_energy - always accept an improvement
    if(np.random.uniform(0,1)<acceptance_probability): # accept
        current_observing_plan = proposed_observing_plan
        current_energy = proposed_energy
        current_schedule = proposed_schedule
        
    temperature = temperature*cooling_rate
    
print_observing_plan(current_observing_plan, current_schedule)



iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10
iteration  11
iteration  12
iteration  13
iteration  14
iteration  15
iteration  16
iteration  17
iteration  18
iteration  19
iteration  20
iteration  21
iteration  22
iteration  23
iteration  24
iteration  25
iteration  26
iteration  27
iteration  28
iteration  29
iteration  30
iteration  31
iteration  32
iteration  33
iteration  34
iteration  35
iteration  36
iteration  37
iteration  38
iteration  39
iteration  40
iteration  41
iteration  42
iteration  43
iteration  44
iteration  45
iteration  46
iteration  47
iteration  48
iteration  49
iteration  50
iteration  51
iteration  52
iteration  53
iteration  54
iteration  55
iteration  56
iteration  57
iteration  58
iteration  59
iteration  60
iteration  61
iteration  62
iteration  63
iteration  64
iteration  65
iteration  66
iteration  67
iteration  68
iteration  69
iteration  70
iteration  71
it